In [ ]:
import numpy as np
import pandas as pd

from os import mkdir
from Bio import SeqIO
from os.path import isdir

def Load_Fasta_Seqs(seq_path):
    d = {}
    fasta_sequences = SeqIO.parse(open(seq_path),'fasta')
    for s in fasta_sequences:
        d[s.name] = str(s.seq)
    return d

def Get_Allele_Id(Subject):
    T = Subject.split('_')
    try:
        return T[0]+"_"+T[1]
    except IndexError:
        return T[0]

In [ ]:
data_path = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
C_Allele_Relative_Abundance_Sep/Filtered_Alignments/Pident_80/'
seq_path = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
C_Allele_Relative_Abundance_Aug/'
out_path = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
C_Allele_Relative_Abundance_Sep/BLAST_Comparison_Ribosomal_Proteins/'

if not isdir(out_path):
    mkdir(out_path)

if not isdir(out_path+'A_Alleles/'):    
    mkdir(out_path+'A_Alleles/')
if not isdir(out_path+'B_Alleles/'):    
    mkdir(out_path+'B_Alleles/')
if not isdir(out_path+'C_Alleles/'):    
    mkdir(out_path+'C_Alleles/')
    

In [ ]:
df_single_cells = pd.read_csv('/Users/harihara/Research-Activities/Data/Hot-Spring/\
Single_Cell_Gabe/sscs_annotated_single_copy_orthogroup_presence.tsv', sep = "\t")
df_single_cells = df_single_cells.rename(columns = {'Unnamed: 0':'Gene-ID'})


In [ ]:
Genes_of_Interest = ['dnaG','frr','pgk','rplA','rplK','rplL','rplM', 'rplS', 'rplT', 
                     'rpsB', 'rpsI', 'rpsJ', 'rpsK','rpsM', 'smpB', 'tsf']
Genes_Sel = []
for g in df_single_cell_summary['Gene-ID'].tolist():
    for f in Genes_of_Interest:
        if g.startswith(f):
            Genes_Sel.append(g)
            
Ribosomal_alleles = df_single_cells[df_single_cells['Gene-ID'].isin(Genes_Sel)]['parent_og_id'].unique()

In [ ]:
df_A = pd.read_excel(data_path+'A_Allele.xlsx', header = [0,1])
df_A = df_A.fillna(0)
df_A = df_A[1:]
df_A['Allele'] = df_A['Sample']['Stat'].apply(Get_Allele_Id)
df_A = df_A.set_index([('Allele','')])
df_A['A_abundances'] = df_A.loc[:, df_A.columns.get_level_values(1)=='Avg_Depth_Coverage'].sum(axis = 1)

df_B = pd.read_excel(data_path+'B_Allele.xlsx', header = [0,1])
df_B = df_B.fillna(0)
df_B = df_B[1:]
df_B['Allele'] = df_B['Sample']['Stat'].apply(Get_Allele_Id)
df_B = df_B.set_index([('Allele','')])
df_B['B_abundances'] = df_B.loc[:, df_B.columns.get_level_values(1)=='Avg_Depth_Coverage'].sum(axis = 1)

df_C = pd.read_excel(data_path+'C_Allele.xlsx', header = [0,1])
df_C = df_C.fillna(0)
df_C = df_C[1:]
df_C['Allele'] = df_C['Sample']['Stat'].apply(Get_Allele_Id)
df_C = df_C.set_index([('Allele','')])
df_C['C_abundances'] = df_C.loc[:, df_C.columns.get_level_values(1)=='Avg_Depth_Coverage'].sum(axis = 1)

df_Total = df_A[['A_abundances']].join(df_B[['B_abundances']]).join(df_C[['C_abundances']])
df_Total = df_Total.fillna(0)
df_Total['Total_Abundance'] = df_Total.sum(axis=1)
df_Total = df_Total[(df_Total['A_abundances'] > 0) & 
                    (df_Total['B_abundances'] > 0) & 
                    (df_Total['C_abundances'] > 0)]
df_Total = df_Total.sort_values(by = 'Total_Abundance', ascending = False)

In [ ]:
A_Alleles_Sel = df_A.loc[Ribosomal_alleles]['Sample']['Stat'].tolist()
B_Alleles_Sel = df_B.loc[Ribosomal_alleles]['Sample']['Stat'].tolist()
C_Alleles_Sel = df_C.loc[Ribosomal_alleles]['Sample']['Stat'].tolist()

In [ ]:
df_C.index.tolist()

In [ ]:
len(df_A.index.tolist())

In [ ]:
Genes_Sel

In [ ]:
df_single_cells[df_single_cells['Gene-ID'].isin(Genes_Sel)]